<a href="https://colab.research.google.com/github/DaryaTereshchenko/AdClick/blob/main/Click_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pprint import pprint
import re

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import IntegerLookup, Concatenate, Flatten


In [3]:
# load data
from urllib.request import urlretrieve
import os

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

d10m_url = "https://home.ipipan.waw.pl/sj/TIB_PAN_Adv/AdClick/D10M.tsv.gz"
download(d10m_url,'D10M.tsv.gz')
print("All the files are downloaded")

# d100k_url = "https://home.ipipan.waw.pl/sj/TIB_PAN_Adv/AdClick/D100k.tsv.gz"
# download(d100k_url,'D100k.tsv.gz')
# print("All the files are downloaded")

Download file... D10M.tsv.gz ...
File downloaded
All the files are downloaded


In [4]:
ds = tf.data.experimental.make_csv_dataset("D10M.tsv.gz",
                                           field_delim="\t",
                                           compression_type="GZIP",
                                           num_epochs=1,
                                           batch_size=1024,  
                                           label_name="Click", shuffle=False)


In [5]:
def get_dataset_partitions_tf(ds, ds_size=10000000, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [6]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(ds)

In [7]:
train = train_ds.unbatch()

needed_vocab = {"AdKeyword_tokens": [], "AdDescription_tokens": [], "Query_tokens": []}

for dic in train.as_numpy_iterator():
  for k, v in dic[0].items():
    if k in needed_vocab.keys():
      needed_vocab[k].append(v)


In [8]:
def transform_to_list(parameter):
  list_of_tokens = list(map(lambda x: x.replace(b"|", b" ").split(b" "), parameter))
  flatten_list = [item for subl in list_of_tokens for item in subl]
  return np.asarray(flatten_list)


In [9]:
keyword_tokens = transform_to_list(needed_vocab["AdKeyword_tokens"])
description_tokens = transform_to_list(needed_vocab["AdDescription_tokens"])
query_tokens = transform_to_list(needed_vocab["Query_tokens"])
total_vocab = np.unique(np.concatenate((keyword_tokens, description_tokens, query_tokens), axis=0))

In [10]:
print(len(description_tokens))
print(len(keyword_tokens))
print(len(query_tokens))
print(len(total_vocab))

213222057
21087925
30146708
221391


In [11]:
maxlen = 128
max_features = 221395 # max word number

In [12]:
def split_on_slash(input_data):
  return tf.strings.regex_replace(input_data, "\|", " ")

In [61]:
def build_model1():
  inputs = {
      "Position": Input(shape=(), dtype=tf.int32),
      "Age": Input(shape=(), dtype=tf.int32),
      "Depth": Input(shape=(), dtype=tf.int32),
      "Gender": Input(shape=(), dtype=tf.int32),
      "AdvertiserId": Input(shape=(), dtype=tf.int32),
      "AdDescription_tokens": Input(shape=(), dtype=tf.string),
      "Query_tokens": Input(shape=(), dtype=tf.string),
      "AdKeyword_tokens": Input(shape=(), dtype=tf.string)}

  # Integer part
  input_age = IntegerLookup(vocabulary=[1,2,3,4,5,6], output_mode="one_hot", num_oov_indices=0)(inputs["Age"])
  ci_p = IntegerLookup(vocabulary=[1,2,3], output_mode="one_hot", num_oov_indices=0)(inputs["Position"])
  ci_d = IntegerLookup(vocabulary=[1,2,3], output_mode="one_hot", num_oov_indices=0)(inputs["Depth"])
  ci_g = IntegerLookup(vocabulary=[0,1,2], output_mode="one_hot", num_oov_indices=0)(inputs["Gender"])
  encoded = Concatenate()([ci_p, input_age, ci_d, ci_g])
  layer = tf.keras.layers.Hashing(num_bins=1024, output_mode="one_hot")(inputs["AdvertiserId"])

  conc = tf.keras.layers.concatenate([layer, encoded])
  num = Dropout(rate=0.3)(conc)
  num = Dense(128)(num)
  num = Flatten()(num)

# String part
  description = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode="int", pad_to_max_tokens=maxlen, standardize=split_on_slash, output_sequence_length=maxlen, vocabulary=total_vocab)(inputs["AdDescription_tokens"])
  x = tf.keras.layers.Embedding(max_features, output_dim=10, input_length=maxlen)(description)
  
  title = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode="int", pad_to_max_tokens=maxlen, standardize=split_on_slash, output_sequence_length=maxlen, vocabulary=total_vocab)(inputs["Query_tokens"])
  y = tf.keras.layers.Embedding(max_features, output_dim=10, input_length=maxlen)(title)

  keywords = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode="int", pad_to_max_tokens=maxlen, standardize=split_on_slash, output_sequence_length=maxlen, vocabulary=total_vocab)(inputs["AdKeyword_tokens"])
  o = tf.keras.layers.Embedding(max_features, output_dim=10, input_length=maxlen)(keywords)

  c = Concatenate()([x, y, o])
  c = tf.keras.layers.SpatialDropout1D(0.3)(c)
  c = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(c)
  c = tf.keras.layers.GlobalAveragePooling1D()(c)
  t = Dropout(rate=0.3)(c)
  # t = Dense(128)(t)
  t = Flatten()(t)

  z = Concatenate()([t, num])
  z = tf.keras.layers.Dense(64, activation='relu')(z)
  z = tf.keras.layers.BatchNormalization()(z)
  z = tf.keras.layers.Dense(1, activation='sigmoid')(z)
  

  model = Model(inputs=inputs, outputs=z)
  model.compile(optimizer="Adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
          metrics=["AUC"], run_eagerly=True)
  return model

In [ ]:
model = build_model1()
model.summary()
tf.keras.utils.plot_model(model)


In [15]:
model.fit(train_ds.unbatch().batch(1024), epochs=12, verbose=1, validation_data=val_ds.unbatch().batch(1024))

Epoch 1/12


/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['DisplayURL', 'AdId', 'UserID', 'AdTitle_tokens'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


9766/9766 [==============================] - 1441s 132ms/step - loss: 0.1805 - auc: 0.6859
Epoch 2/12
9766/9766 [==============================] - 1298s 119ms/step - loss: 0.1726 - auc: 0.7235
Epoch 3/12
9766/9766 [==============================] - 1311s 120ms/step - loss: 0.1708 - auc: 0.7366
Epoch 4/12
9766/9766 [==============================] - 1315s 120ms/step - loss: 0.1694 - auc: 0.7455
Epoch 5/12
9766/9766 [==============================] - 1319s 120ms/step - loss: 0.1684 - auc: 0.7520
Epoch 6/12
9766/9766 [==============================] - 1322s 121ms/step - loss: 0.1675 - auc: 0.7574
Epoch 7/12
9766/9766 [==============================] - 1323s 121ms/step - loss: 0.1667 - auc: 0.7619
Epoch 8/12
9766/9766 [==============================] - 1321s 121ms/step - loss: 0.1660 - auc: 0.7653
Epoch 9/12
9766/9766 [==============================] - 1323s 121ms/step - loss: 0.1655 - auc: 0.7670
Epoch 10/12
9766/9766 [==============================] - 1323s 121ms/step - loss: 0.1648 - au

In [52]:
from urllib.request import urlretrieve
import os

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

d5m_url = "https://home.ipipan.waw.pl/sj/TIB_PAN_Adv/AdClick/D5M_test_x.tsv.gz"
download(d5m_url,'D5M_test_x.tsv.gz')
print("All the files are downloaded")

All the files are downloaded


In [59]:
test_data = tf.data.experimental.make_csv_dataset("D5M_test_x.tsv.gz",
                                           field_delim="\t",
                                           compression_type="GZIP",
                                           num_epochs=1,
                                           batch_size=1024,  
                                           label_name="Click", 
                                           column_defaults = (tf.float32, tf.int64, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32,
                                                              tf.string, tf.string, tf.string, tf.string, tf.string),
                                           shuffle=False, ignore_errors=True)


In [45]:
# test_data = tf.data.experimental.make_csv_dataset("D5M_test_x.tsv.gz",
#                                            field_delim="\t",
#                                            compression_type="GZIP",
#                                            num_epochs=1,
#                                            batch_size=1024,
#                                            label_name="Click",
#                                            column_defaults = (tf.float32, tf.int64, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32,
#                                                               tf.string, tf.string, tf.string, tf.string, tf.string),
#                                            shuffle=False,
#                                            ignore_errors=True)


In [42]:
pprint(test_data.element_spec)

(OrderedDict([('DisplayURL',
               TensorSpec(shape=(None,), dtype=tf.int64, name=None)),
              ('AdId', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('AdvertiserId',
               TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('Depth', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('Position',
               TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('UserID', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('Gender', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('Age', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('AdKeyword_tokens',
               TensorSpec(shape=(None,), dtype=tf.string, name=None)),
              ('AdTitle_tokens',
               TensorSpec(shape=(None,), dtype=tf.string, name=None)),
              ('AdDescription_tokens',
               TensorSpec(shape=(None,), dtype=tf.string, na

In [62]:
y_pred = model.predict(test_data.unbatch().batch(256).take(1))

ValueError: ignored

In [ ]:
y_pred